In [86]:
import pandas as pd
import numpy as np 
import ast


In [87]:
# Define data paths constants
PLOT_DATA_PATH = "../data/plot_summaries.txt"
MOVIE_DATA_PATH = "../data/movie.metadata.tsv"
CLUSTER_NAME_DATA_PATH = "../data/name.clusters.txt"
CHARACTER_DATA_PATH = "../data/character.metadata.tsv"

In [88]:
"""
Function to parse dictionary-like strings in the file and separate keys and values
"""
def parse_dict_column(column):
    parsed_keys = []
    parsed_values = []
    
    for item in column:
        # Convert string representation of dictionary to actual dictionary
        item_dict = ast.literal_eval(item)
        parsed_keys.append(", ".join(item_dict.keys()))
        parsed_values.append(", ".join(item_dict.values()))
    
    return parsed_keys, parsed_values

In [ ]:
"""
Function to load and clean movie metadata, returns a dataframe with cleaned movie data
"""
def load_and_clean_movie_data():
    # Load the movie metadata
    df_movie_metadata = pd.read_csv(
        MOVIE_DATA_PATH, sep='\t', header=None, 
        names=[
            'Wikipedia_movie_ID', 'Freebase_movie_ID', 'Movie_name', 
            'Movie_release_date', 'Movie_box_office_revenue', 'Movie_runtime',
            'Movie_languages_(Freebase ID:name tuples)', 'Movie_countries_(Freebase ID:name tuples)',
            'Movie_genres_(Freebase ID:name tuples)'
        ]
    )
    
    # Parse 'languages', 'countries', and 'genres' columns
    df_movie_metadata['id_movie_languages'], df_movie_metadata['Movie_languages'] = parse_dict_column(df_movie_metadata['Movie_languages_(Freebase ID:name tuples)'])
    df_movie_metadata['id_Movie_countries'], df_movie_metadata['Movie_countries'] = parse_dict_column(df_movie_metadata['Movie_countries_(Freebase ID:name tuples)'])
    df_movie_metadata['id_Movie_genres'], df_movie_metadata['Movie_genres'] = parse_dict_column(df_movie_metadata['Movie_genres_(Freebase ID:name tuples)'])

    # Convert dates to datetime and extract the year
    df_movie_metadata['Movie_release_date'] = pd.to_datetime(df_movie_metadata['Movie_release_date'], errors='coerce').dt.year

    # Select and rename the columns as required
    cleaned_df_movie_metadata = df_movie_metadata[[
        'Wikipedia_movie_ID', 'Freebase_movie_ID', 'Movie_name', 'Movie_release_date', 
        'Movie_box_office_revenue', 'Movie_runtime', 'id_movie_languages', 
        'Movie_languages', 'id_Movie_countries', 'Movie_countries', 
        'id_Movie_genres', 'Movie_genres'
    ]]

    # Drop unwanted id columns
    columns_to_drop = ['id_movie_languages', 'id_Movie_genres', 'id_Movie_countries']
    cleaned_df_movie_metadata = cleaned_df_movie_metadata.drop(columns=columns_to_drop)

    # Convert floats of box office and years to nullable integers, keeping NaNs as np.nan
    cleaned_df_movie_metadata['Movie_box_office_revenue'] = pd.to_numeric(cleaned_df_movie_metadata['Movie_box_office_revenue'], errors='coerce')
    cleaned_df_movie_metadata['Movie_release_date'] = pd.to_numeric(cleaned_df_movie_metadata['Movie_release_date'], errors='coerce')

    #need to drop a line that has some weird encodings 
    df = df.applymap(lambda x: x.encode('utf-8', 'ignore').decode('utf-8') if isinstance(x, str) else x)
    df.to_csv('filename.csv', index=False, encoding='utf-8')

    # Replace any <NA> with np.nan for uniform NaNs
    cleaned_df_movie_metadata = cleaned_df_movie_metadata.replace({pd.NA: np.nan})

    return cleaned_df_movie_metadata

df_movie_metadata = load_and_clean_movie_data()
df_movie_metadata.sample(2)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
79681,35509800,/m/0j9kczf,Night Eyes 4: Fatal Passion,1996.0,NaN,101.0,English Language,United States of America,"Thriller, Drama, Erotica, Adult"
63657,10404153,/m/02qbvf2,Notebook,2006.0,NaN,143.0,Malayalam Language,India,"World cinema, Musical, Drama, Romance Film, Bo..."


In [90]:
df_movie_metadata.iloc[1820:1830]

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
1820,3443975,/m/09cr2x,Transfer,NaN,NaN,NaN,English Language,Canada,"Short Film, Surrealism"
1821,2430019,/m/07ch32,Tokyo Joe,NaN,NaN,88.0,English Language,United States of America,"Thriller, Crime Fiction, Romantic drama, Crime..."
1822,15036863,/m/03h5h68,Bodyguard,1948.0,NaN,62.0,English Language,United States of America,"Thriller, Crime Fiction, Detective, Black-and-..."
1823,28858020,/m/0ddh66w,"Carmen, la de Triana",NaN,NaN,110.0,Spanish Language,"Spain, Germany",Drama
1824,34666140,/m/0j240f7,Enga Ooru Mappillai,NaN,NaN,NaN,Tamil Language,India,
1826,9002230,/m/027s_mk,Arroz con leche,1950.0,NaN,79.0,Spanish Language,Argentina,Comedy
1827,4703636,/m/0cj5ry,The Dark,NaN,NaN,93.0,"Welsh Language, English Language","United Kingdom, Germany","Thriller, Mystery, Horror"
1828,23288456,/m/065y6l7,Blind Love,2005.0,NaN,65.0,,Japan,"Romance Film, Japanese Movies, Erotica, Pornog..."
1829,23367560,/m/06w9nvb,Heaven Is Round the Corner,1944.0,NaN,103.0,English Language,United Kingdom,Musical
1830,28042876,/m/0cmf0m0,Puss in Boots,2011.0,554709226.0,90.0,English Language,United States of America,"Comedy, Fantasy, Adventure, Animation"


In [93]:
 #make csv temporary
df_movie_metadata.to_csv('filename.csv', index=False, encoding='ISO-8859-1')


UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 24: ordinal not in range(256)